In [1]:
import re
import pickle
import os
from IPython.display import display, HTML

In [2]:
from reader import Reader
src_file = "../../processed_pdfs/sample1/sample1.xml"
r = Reader({"src": src_file})
blocks, kv, text = r.get_content()
content = {"texts": text, "blocks": blocks, "kv": kv}
with open("tmp.pkl", "wb") as fi:
    pickle.dump(content, fi)

Number of Pages:  2


In [3]:
from extractor import Extractor
M = Extractor()

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Total number of trainable parameters: 788673
INFO:tensorflow:Restoring parameters from models/FRC/model/model_60000.ckpt


In [17]:
file_name = "tmp.pkl"
with open(file_name, "rb") as fi:
    content = pickle.load(fi)
M.set_defs("./defs.json")    
M.set_content(content)
df = M.extract()
display(df)

,Name,Value,Type,Method,Region,Source,Page
0,Report Format,Restricted Appraisal Report,None,lookup,None,None,0
1,Date of Report,"11th January, 2018",entity,search,key,"Date of Report: 11th January, 2018",0
2,Date of Inspection,"10th January, 2018",entity,search,key,"Date of Inspection: 10th January, 2018",0
3,File Reference #,None,None,None,None,None,None
4,Asset #,None,None,regex,None,None,None
5,Work Request #,None,None,regex,None,None,None
6,Tracking #,None,None,regex,None,None,None
7,Interest Appraised,None,None,None,None,None,None
8,Intended Use,financial assessment,text,search,key,Intended Use This report is prepared for the p...,1
9,Intended User,Citibank N.A,entity,search,key,Intended User Citibank N.A,1
